# <font color='#002726'> Data Science em Produção </font>
    
=-=- ROSSMANN - STORE SALES PREDICTION -=-=

# <font color='#36013F'> 0. Imports </font>

In [ ]:
import inflection  # helper function

import numpy as np
import pandas as pd
import seaborn as sns

## <font color='#1F45FC'> 0.1. Helper Functions </font>

In [ ]:
def snakecase(list_of_names):
    """Returns a list of names in snake case, which refers to the style of  
    writing in which each space is replaced by an underscore (_) character."""
    
    new_list = list(map(inflection.underscore, list_of_names))
    return new_list



## <font color='#1F45FC'> 0.2. Loading Data </font>

In [ ]:
# loading historical data - including Sales
df_sales_raw = pd.read_csv('../raw_data/train.csv', low_memory=False)

# loading information about the stores
df_store_raw = pd.read_csv('../raw_data/store.csv', low_memory=False)

### <font color='#F37126'> Data Fields </font>

**Most of the fields are self-explanatory. The following are descriptions for those that aren't.**

- **Id** - an Id that represents a (Store, Date) duple within the test set;
- **Store** - a unique Id for each store;
- **Sales** - the turnover for any given day (this is what you are predicting);
- **Customers** - the number of customers on a given day;
- **Open** - an indicator for whether the store was open: 0 = closed, 1 = open;
- **StateHoliday** - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None;
- **SchoolHoliday** - indicates if the (Store, Date) was affected by the closure of public schools;
- **StoreType** - differentiates between 4 different store models: a, b, c, d;
- **Assortment** - describes an assortment level: a = basic, b = extra, c = extended;
- **CompetitionDistance** - distance in meters to the nearest competitor store;
- **CompetitionOpenSince[Month/Year]** - gives the approximate year and month of the time the nearest competitor was opened;
- **Promo** - indicates whether a store is running a promo on that day;
- **Promo2** - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating;
- **Promo2Since[Year/Week]** - describes the year and calendar week when the store started participating in Promo2;
- **PromoInterval** - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store.

# <font color='#36013F'> 1. Descriptive Data Analysis </font>

## <font color='#1F45FC'> 1.0. Dataframe in Progress Backup </font>

In [ ]:
df1_store = df_store_raw.copy()

df1_sales = df_sales_raw.copy()

## <font color='#1F45FC'> 1.1. Column Renaming </font>

In [ ]:
# renaming df1_store column names
df1_store.columns = snakecase(df1_store.columns)

# renaming df1_sales column names
df1_sales.columns = snakecase(df1_sales.columns)

## <font color='#1F45FC'> 1.2. Data Dimension </font>

In [ ]:
print(f'Store Dataframe - Number of Rows: {df1_store.shape[0]}. \nStore Dataframe - Number of Columns: {df1_store.shape[1]}.', end='\n\n')
print(f'Sales Dataframe - Number of Rows: {df1_sales.shape[0]}. \nSales Dataframe - Number of Columns: {df1_sales.shape[1]}.')

## <font color='#1F45FC'> 1.3. Data Types </font>

In [ ]:
# Store dataframe data types
df1_store.dtypes

In [ ]:
# Sales dataframe data types
df1_sales.dtypes

In [ ]:
# setting date column as datetime type
df1_sales['date'] = pd.to_datetime(df1_sales['date'])

df1_sales.dtypes

## <font color='#1F45FC'> 1.4. Check NA </font>

In [ ]:
# checking NA in df1_sales - no NAs
df1_sales.isna().sum()

In [ ]:
# checking NA in d1_store
df1_store.isna().sum()

## <font color='#1F45FC'> 1.5. Fillout NA </font>

**Store dataframe - df_store**

    competition_distance              3

    competition_open_since_month    354
    competition_open_since_year     354

    promo2_since_week               544
    promo2_since_year               544
    promo_interval                  544

In [ ]:
# competition_distance

# maximun distance x 2
max_dist_x_2 = df1_store['competition_distance'].max() * 2

# assuming competitors are twice as far away as the greatest distance found
df1_store['competition_distance'] = df1_store['competition_distance'].apply(lambda x: max_dist_x_2 if np.isnan(x) else x)

In [ ]:
# competition_open_since_year

ax = sns.boxplot(x='competition_open_since_year', data=df1_store)

In [ ]:
# competition_open_since_year

# frequency per year of existing competition_open_since_year data
frequency = df1_store['competition_open_since_year'].value_counts(
                                                     normalize=True).reset_index().rename(
                                                     columns={'index': 'year', 
                                                              'competition_open_since_year': 'percent'})

# True/False missing/Null Series
missing = df1_store['competition_open_since_year'].isna()

# Using Numpy's random.choice to fill out missing data based on the frequency of existing info
df1_store.loc[missing,'competition_open_since_year'] = np.random.choice(frequency.year, 
                                                                        size=len(df1_store[missing]), 
                                                                        p=frequency.percent)

In [ ]:
# competition_open_since_month

# frequency per month of existing competition_open_since_month data
frequency = df1_store['competition_open_since_month'].value_counts(
                                                          normalize=True).reset_index().rename(
                                                          columns={'index': 'month', 
                                                                   'competition_open_since_month': 'percent'})

# True/False missing/Null Series
missing = df1_store['competition_open_since_month'].isna()

# Using Numpy's random.choice to fill out missing data based on the frequency of existing info
df1_store.loc[missing,'competition_open_since_month'] = np.random.choice(frequency.month, 
                                                                         size=len(df1_store[missing]), 
                                                                         p=frequency.percent)